# DS18B20: Bring up and basic usage of the temperature sensor

*Important:*  
The **DS18B20** is a digital thermometer and provides 9-bit to 12-bit Celsius temperature measurements. It has an alarm function with nonvolatile user-programmable upper and lower trigger points. The DS18B20 communicates over a **1-Wire** bus that by definition requires only one data line (and ground) for communication with a central microprocessor.

- Data sheet: [https://datasheets.maximintegrated.com/en/ds/DS18B20.pdf](https://datasheets.maximintegrated.com/en/ds/DS18B20.pdf)

## Variant 1: Using USB interface board Temp-Sensor-Tester

The communication with the the temperature sensor(s) DS18B20 takes place via an USB interface board *Temp-Sensor-Tester* from *DIAMEX GmbH*. This interface board displays the connected DS18B20 temperature sensor(s) and their readings as a Human Interface Device (HID) device.

![USB-Temperatur-Sensor-Tester für DS18B20, Rev. C (Diamex)](images/Diamex_TempSensorBoard.jpg)  
USB-Temperatur-Sensor-Tester für DS18B20, Rev. C (Diamex)


Gerneral bringup information:
- USB-Temperature-Sensor-Tester for DS18B20, Rev. C: [https://www.diamex.de/dxshop/USB-Temperatur-Sensor-Tester-fuer-DS18B20-Rev-C](https://www.diamex.de/dxshop/USB-Temperatur-Sensor-Tester-fuer-DS18B20-Rev-C) or [https://www.led-genial.de/USB-Temperatur-Sensor-Tester-fuer-DS18B20-Rev-C](https://www.led-genial.de/USB-Temperatur-Sensor-Tester-fuer-DS18B20-Rev-C)
- Manual and software: [https://www.diamex.de/dxshop/mediafiles//Sonstiges/TempSensorTester.zip](https://www.diamex.de/dxshop/mediafiles//Sonstiges/TempSensorTester.zip) or [https://www.led-genial.de/mediafiles//Sonstiges/TempSensorTester.zip](https://www.led-genial.de/mediafiles//Sonstiges/TempSensorTester.zip)
- Diamex Temp-Sensor-Tester für DS18B20 unter Ubuntu Linux 20.04 (in german): [https://www.docollipics.de/diamex-temp-sensor-tester-fuer-ds18b20-unter-ubuntu-linux-20-04/](https://www.docollipics.de/diamex-temp-sensor-tester-fuer-ds18b20-unter-ubuntu-linux-20-04/)
- **tempsense** (command line utilty): [https://github.com/kybernetyk/tempsense](https://github.com/kybernetyk/tempsense)

Python libraries under consideration:
- Python **hidapi** library: [https://pypi.org/project/hidapi/}(https://pypi.org/project/hidapi/) <span style="color:green">[works]</span>
- Python **hid** library: [https://pypi.org/project/hid/](https://pypi.org/project/hid/) <span style="color:red">[does NOT work]</span>

Eventually, the communication with the HID device is realized with the cython package **hidapi** because of the good test results.

### Installation of required development libraries

```
$ sudo apt install python-dev libusb-1.0-0-dev libudev-dev
```

### Installation of required Python libraries

The installation takes place in a Python environment:
```
$ source ~/jupyter-env/bin/activate
$ pip install hidapi
```

### Udev rule

To be able to address the device, you need to create a rule file in udev rules directory. Make sure that your current user belongs to the *plugdev* group.
```
$ sudo echo 'SUBSYSTEM=="usb", ATTRS{idVendor}=="16c0", ATTRS{idProduct}=="0480", MODE="0660", GROUP="plugdev"' > /etc/udev/rules.d/99-tempsensor.rules
```

Activate the new rule by calling
```
$ sudo udevadm control --reload-rules
```

### Sample usage of the *hidapi* library

Use the ```enumerate()``` function to return information about accessible HID devices on USB:

In [15]:
import hid

for device_dict in hid.enumerate():
    keys = list(device_dict.keys())
    keys.sort()
    for key in keys:
        print("%s : %s" % (key, device_dict[key]))
    print()

interface_number : 0
manufacturer_string : DIAMEX GmbH
path : b'1-1.4:1.0'
product_id : 1152
product_string : Temp-Sensor-Tester
release_number : 256
serial_number : 
usage : 0
usage_page : 0
vendor_id : 5824



The details about a HID device can be printed with following code:

In [20]:
vid = 0x16c0 # Change it for your device
pid = 0x0480 # Change it for your device

h = hid.device()
h.open(vid, pid)
#h.open_path(b'1-1.4:1.0')

print("Manufacturer: %s" % h.get_manufacturer_string())
print("Product: %s" % h.get_product_string())
print("Serial No: %s" % h.get_serial_number_string())

#h.close()

Manufacturer: DIAMEX GmbH
Product: Temp-Sensor-Tester
Serial No: Љ


In [38]:
connected_with = '{:s} ({:s}) with idVendor={:#06x} and idProduct={:#06x} over USB'.format(h.get_product_string(), h.get_manufacturer_string(), vid, pid)
connected_with

'Temp-Sensor-Tester (DIAMEX GmbH) with idVendor=0x16c0 and idProduct=0x0480 over USB'

### Automation in a sample program

In [59]:
import hid
import time

INTERVAL = 1.0
VID = 0x16c0 # vendor id of usb device
PID = 0x0480 # product id of usb device

try:
    h = hid.device()
    h.open(VID, PID)
    print("Device Info:")
    print("\t Manufacturer: %s" % h.get_manufacturer_string())
    print("\t Product: %s" % h.get_product_string())

except Exception as ex:
    print("Connecting with the device raised the error: '{}'".format(ex))
    raise

while True:
    try:
        d = h.read(64)

        if d:
            sensor_id = ''
            # get sensor ID with the bytes 8 to 16 from character buffer
            for i in range(8, 16, 1):
                # convert integer from byte list to string with padding
                hex_string = '{:02x}'.format(d[i])
                sensor_id = sensor_id + hex_string + ' '

            # combine high and low byte of temperature value and convert to float
            temp = float(d[5] << 8 | d[4]) / 10

            print("<{:s}> Sensor: {:d} of {:d}, Temperature: {:.2f} °C, ID: {}".format(time.strftime('%H:%M:%S'), d[1], d[0], temp, sensor_id))

        else:
            print("Could not read from device!")
            break
    
        time.sleep(INTERVAL)
        
    except:
        print("Keyboard Interrupt ^C detected.")
        print("Bye.")
        h.close()
        break

Device Info:
	 Manufacturer: DIAMEX GmbH
	 Product: Temp-Sensor-Tester
<00:27:40> Sensor: 1 of 2, Temperature: 24.20 °C, ID: 28 64 8e 75 d0 01 3c 81 
<00:27:41> Sensor: 2 of 2, Temperature: 21.80 °C, ID: 28 84 28 e2 2c 20 01 10 
<00:27:42> Sensor: 1 of 2, Temperature: 24.20 °C, ID: 28 64 8e 75 d0 01 3c 81 
<00:27:43> Sensor: 2 of 2, Temperature: 21.80 °C, ID: 28 84 28 e2 2c 20 01 10 
<00:27:44> Sensor: 1 of 2, Temperature: 24.30 °C, ID: 28 64 8e 75 d0 01 3c 81 
<00:27:45> Sensor: 2 of 2, Temperature: 21.80 °C, ID: 28 84 28 e2 2c 20 01 10 
<00:27:46> Sensor: 1 of 2, Temperature: 24.20 °C, ID: 28 64 8e 75 d0 01 3c 81 
<00:27:47> Sensor: 2 of 2, Temperature: 21.80 °C, ID: 28 84 28 e2 2c 20 01 10 
<00:27:48> Sensor: 1 of 2, Temperature: 24.30 °C, ID: 28 64 8e 75 d0 01 3c 81 
<00:27:49> Sensor: 2 of 2, Temperature: 21.80 °C, ID: 28 84 28 e2 2c 20 01 10 
<00:27:50> Sensor: 1 of 2, Temperature: 24.20 °C, ID: 28 64 8e 75 d0 01 3c 81 
<00:27:51> Sensor: 2 of 2, Temperature: 21.80 °C, ID: 28 84 

### Using the wrapper class 'DS18B20_over_USB'

The new wrapper class **DS18B20_over_USB** in the python file ```DS18B20_class.py``` implements the communication with the the temperature sensor DS18B20 via USB interface board *Temp-Sensor-Tester* from *DIAMEX GmbH*. This interface board displays the connected DS18B20 temperature sensor(s) and their readings as a Human Interface Device (HID) device. The communication with the HID device is realized with the cython package *hidapi*.

In [1]:
# import wrapper class DS18B20_class from python file DS18B20_over_USB.py
from DS18B20_class import DS18B20_over_USB

In [2]:
# create new device object for the USB interface board
if_board = DS18B20_over_USB(vid = 0x16c0, pid = 0x0480)

In [3]:
# read connection state to the USB interface board
if_board.status

'Connected'

In [4]:
# read connection details
if_board.connected_with

'Temp-Sensor-Tester (DIAMEX GmbH) with idVendor=0x16c0 and idProduct=0x0480 over USB'

In [18]:
# close the connection to the USB interface board
if_board.closeConnection()

In [4]:
# open the connection again
if_board.openConnection(vid = 0x16c0, pid = 0x0480)

In [7]:
# get an array of sensor id(s)
if_board.getSensorIDs()

['28 64 8e 75 d0 01 3c 81', '28 84 28 e2 2c 20 01 10']

In [15]:
# read sensor values to dictionary
temp_dict = if_board.getTemperature_dict()
temp_dict

{'28 84 28 e2 2c 20 01 10': 21.7, '28 64 8e 75 d0 01 3c 81': 22.1}

In [16]:
temp_dict['28 84 28 e2 2c 20 01 10']

21.7

In [17]:
temp_dict['28 64 8e 75 d0 01 3c 81']

22.1

In [65]:
import time

# import wrapper class DS18B20_class from python file DS18B20_over_USB.py
from DS18B20_class import DS18B20_over_USB

INTERVAL = 1.0
VID = 0x16c0 # vendor id of usb device
PID = 0x0480 # product id of usb device

# create new device object for the USB interface board
if_board = DS18B20_over_USB(VID, PID)

# get an ordered array of sensor id(s)
sensor_ids_array = if_board.getSensorIDs()
sensor_ids_cnt = len(sensor_ids_array)

while True:
    try:
        temp_dict = if_board.getTemperature_dict()

        for i in range(0, sensor_ids_cnt, 1):
            sensor_id = sensor_ids_array[i]
            sensor_temp = temp_dict[sensor_id]

            print("<{:s}> Sensor: {:d} of {:d}, Temperature: {:.1f} °C, ID: {}".format(time.strftime('%H:%M:%S'), i+1, sensor_ids_cnt, sensor_temp, sensor_id))

        time.sleep(INTERVAL)
        
    except:
        print("Keyboard Interrupt ^C detected.")
        print("Bye.")
        # close the connection to the USB interface board
        if_board.closeConnection()
        break

<00:37:30> Sensor: 1 of 2, Temperature: 24.1 °C, ID: 28 64 8e 75 d0 01 3c 81
<00:37:30> Sensor: 2 of 2, Temperature: 21.7 °C, ID: 28 84 28 e2 2c 20 01 10
<00:37:33> Sensor: 1 of 2, Temperature: 24.1 °C, ID: 28 64 8e 75 d0 01 3c 81
<00:37:33> Sensor: 2 of 2, Temperature: 21.7 °C, ID: 28 84 28 e2 2c 20 01 10
<00:37:36> Sensor: 1 of 2, Temperature: 24.1 °C, ID: 28 64 8e 75 d0 01 3c 81
<00:37:36> Sensor: 2 of 2, Temperature: 21.7 °C, ID: 28 84 28 e2 2c 20 01 10
<00:37:39> Sensor: 1 of 2, Temperature: 24.1 °C, ID: 28 64 8e 75 d0 01 3c 81
<00:37:39> Sensor: 2 of 2, Temperature: 21.7 °C, ID: 28 84 28 e2 2c 20 01 10
<00:37:42> Sensor: 1 of 2, Temperature: 24.1 °C, ID: 28 64 8e 75 d0 01 3c 81
<00:37:42> Sensor: 2 of 2, Temperature: 21.7 °C, ID: 28 84 28 e2 2c 20 01 10
<00:37:45> Sensor: 1 of 2, Temperature: 24.1 °C, ID: 28 64 8e 75 d0 01 3c 81
<00:37:45> Sensor: 2 of 2, Temperature: 21.7 °C, ID: 28 84 28 e2 2c 20 01 10
<00:37:48> Sensor: 1 of 2, Temperature: 24.1 °C, ID: 28 64 8e 75 d0 01 3c 81

## Variant 2: Directly through the GPIOs of the RPi

The temperature sensor(s) DS18B20 are directly connected to the GPIOs of the Raspberry Pi. In the following it is assumed that the temperature sensor(s) DS18B20 is connected with the data pin to GPIO 4 of the Raspberry Pi. The separately required pull-up resistor can be omitted if a small interface board especially for 1-wire bus is connected in between. The following picture shows the 1-wire interface board I use.

![1-wire interface board with internal pullup resistors](images/1-wire_InterfaceBoard.jpg)  
1-wire interface board with internal pullup resistors


### Activation of the 1-wire bus on the Raspberry Pi

The activation of the 1-wire bus can be done either with the tool *raspi-config* or by manual editing of the file ```/boot/config.txt```.

When using the *raspi-config* tool, however, ```/boot/config.txt``` should still be checked and the entry for 1-wire adjusted as follows if necessary:

```
# configure 1-wire prtocol with data input on gpio pin 4
dtoverlay=w1-gpio,gpiopin=4
```

After a reboot, the following kernel modules should have been loaded for the 1-wire bus:

```
$ lsmod | grep -i w1
w1_therm               28672  0
w1_gpio                16384  0
wire                   36864  2 w1_gpio,w1_therm
```

The DS18B20 sensors connected to GPIO 4 are addressable in the ```/sys``` file system below ```/sys/bus/w1/devices/```. A separate subdirectory structure with the respective sensor ID is created for each detected sensor.

For example, you can read the current temperature in the file ```/sys/bus/w1/devices/28-01202ceb55a3/temperature``` (divide value by 1000). Since the sensor supports a variable resolution of 9 to 12 bits, one can write the desired resolution into the file ```/sys/bus/w1/devices/28-01202ceb55a3/resolution```.

Further literature can be found here, for example:

- Temperatur-Sensor DS1820 am Raspberry Pi mit Python: [https://webnist.de/temperatur-sensor-ds1820-am-raspberry-pi-mit-python/](https://webnist.de/temperatur-sensor-ds1820-am-raspberry-pi-mit-python/)
- Raspberry Pi: 1-Wire Temperatursensor DS1820: [http://www.netzmafia.de/skripten/hardware/RasPi/Projekt-Onewire/index.html](http://www.netzmafia.de/skripten/hardware/RasPi/Projekt-Onewire/index.html)
- Raspberry Pi – Temperaturmessung mit DS18B20 1-Wire Sensor: [http://www.it-adviser.net/raspberry-pi-temperaturmessung-mit-ds18b20-1-wire-sensor/](http://www.it-adviser.net/raspberry-pi-temperaturmessung-mit-ds18b20-1-wire-sensor/)

### Automation in a sample program for only one sensor

In [63]:
import os, sys, time

INTERVAL = 0.2
DEVICE_ID = '28-012116c9fc70'
#DEVICE_ID = '28-01202ceb55a3'

# temperature offset [K]
temp_offset = 0.0

def currentTemperature():
    # Read temperature from 1-wire slave of DS18B20
    # file = open('/sys/bus/w1/devices/28-01202ceb55a3/w1_slave')
    file = open('/sys/bus/w1/devices/'+ DEVICE_ID + '/temperature')
    filecontent = file.read()
    file.close()

    # Get temperature values and convert them
    # stringvalue = filecontent.split("\n")[1].split(" ")[9]
    # temperature = float(stringvalue[2:]) / 1000
    temperature = float(filecontent) / 1000

    # Return temperature
    # ret_val = '%6.4f' % temperature 
    return(temperature)

while True:
    try:
        temp = currentTemperature()
        temp_corr = temp + temp_offset

        print("<{:s}> Temperature: {:.3f} °C".format(time.strftime('%H:%M:%S'), temp_corr))

        time.sleep(INTERVAL)
    
    except:
        print("Keyboard Interrupt ^C detected.")
        print("Bye.")
        break

<00:35:29> Temperature: 22.687 °C
<00:35:30> Temperature: 22.687 °C
<00:35:31> Temperature: 22.687 °C
<00:35:32> Temperature: 22.687 °C
<00:35:33> Temperature: 22.687 °C
<00:35:34> Temperature: 22.687 °C
<00:35:35> Temperature: 22.687 °C
<00:35:37> Temperature: 22.687 °C
<00:35:38> Temperature: 22.687 °C
<00:35:39> Temperature: 22.687 °C
<00:35:40> Temperature: 22.687 °C
<00:35:41> Temperature: 22.687 °C
<00:35:42> Temperature: 22.687 °C
Keyboard Interrupt ^C detected.
Bye.


### Detection of connected sensors

The detection and subsequent readout of multiple DS18B20 sensors requires a recursive search of the directory structures below ```/sys/bus/w1/devices/```.

Literature:

- Mehrere DS1820 Sensoren am Raspberry Pi mit Python einlesen: [http://webnist.de/mehrere-ds1820-sensoren-am-raspberry-pi-mit-python-einlesen/](http://webnist.de/mehrere-ds1820-sensoren-am-raspberry-pi-mit-python-einlesen/)
- List all subdirectories in a directory in Python: [https://www.techiedelight.com/list-all-subdirectories-in-directory-python/](https://www.techiedelight.com/list-all-subdirectories-in-directory-python/)

In [28]:
import os

device_path = '/sys/bus/w1/devices/'
sensor_id_list = []

for obj in os.scandir(device_path):
    # sensor IDs start with '28'
    if obj.is_dir() and obj.name.split("-")[0] == "28":
        sensor_id_list.append(obj.name)

In [29]:
sensor_id_list

['28-012116c9fc70', '28-01202ceb55a3']

### Automation in a sample program for multiple sensors

In [66]:
import os, sys, time

INTERVAL = 0.2

device_path = '/sys/bus/w1/devices/'
sensor_id_list = []
sensor_value_list = []

# temperature offset [K]
temp_offset = 0.0

def detectSensors():
    global sensor_id_list
    
    try:
        for obj in os.scandir(device_path):
            # sensor IDs start with '28'
            if obj.is_dir() and obj.name.split("-")[0] == "28":
                sensor_id_list.append(obj.name)
                
    except Exception as ex:
        print("Searching for sensors raised the error: '{}'".format(ex))
        raise
        

def readoutSensors():
    global sensor_id_list, sensor_value_list
    
    # initialize list
    sensor_value_list = []
    
    # iterate over the sensor ID list
    for sensor_id in sensor_id_list:
        # read temperature from 1-wire slave of DS18B20
        file = open(device_path + sensor_id + '/temperature')
        filecontent = file.read()
        file.close()

        # Get temperature values and convert them
        temperature = float(filecontent) / 1000
        sensor_value_list.append(temperature)

    # return temperature list
    return(sensor_value_list)
            
detectSensors()

while True:
    try:
        readoutSensors()
        
        i = 0
        for sensor_value in sensor_value_list:
            temp_corr = sensor_value + temp_offset
        
            print("<{:s}> Sensor: {:d} of {:d}, Temperature: {:.3f} °C, ID: {}".format(time.strftime('%H:%M:%S'), i+1, len(sensor_id_list), temp_corr, sensor_id_list[i]))
            
            i = i + 1

        time.sleep(INTERVAL)
    
    except:
        print("Keyboard Interrupt ^C detected.")
        print("Bye.")
        break

<00:39:32> Sensor: 1 of 2, Temperature: 22.125 °C, ID: 28-012116c9fc70
<00:39:32> Sensor: 2 of 2, Temperature: 21.687 °C, ID: 28-01202ceb55a3
<00:39:34> Sensor: 1 of 2, Temperature: 22.125 °C, ID: 28-012116c9fc70
<00:39:34> Sensor: 2 of 2, Temperature: 21.687 °C, ID: 28-01202ceb55a3
<00:39:35> Sensor: 1 of 2, Temperature: 22.125 °C, ID: 28-012116c9fc70
<00:39:36> Sensor: 2 of 2, Temperature: 21.687 °C, ID: 28-01202ceb55a3
<00:39:37> Sensor: 1 of 2, Temperature: 22.125 °C, ID: 28-012116c9fc70
<00:39:37> Sensor: 2 of 2, Temperature: 21.750 °C, ID: 28-01202ceb55a3
<00:39:39> Sensor: 1 of 2, Temperature: 22.125 °C, ID: 28-012116c9fc70
<00:39:39> Sensor: 2 of 2, Temperature: 21.750 °C, ID: 28-01202ceb55a3
<00:39:41> Sensor: 1 of 2, Temperature: 22.125 °C, ID: 28-012116c9fc70
<00:39:41> Sensor: 2 of 2, Temperature: 21.687 °C, ID: 28-01202ceb55a3
<00:39:43> Sensor: 1 of 2, Temperature: 22.125 °C, ID: 28-012116c9fc70
<00:39:43> Sensor: 2 of 2, Temperature: 21.687 °C, ID: 28-01202ceb55a3
<00:39